In [114]:
import pandas as pd
# from scipy.stats import pearsonr

#create DataFrame
df = pd.read_csv('2022-jd.csv')


In [115]:
highest = {}

cols = df.columns
cols = cols.drop('date').drop('bottles drank').drop('times peed').drop('time slept').drop('time awaked').drop('total sleep time').drop('effectiveness (Am I SPENDING the RIGHT TIME)').drop('Sleep Min').drop('Sleep Hr').drop('Awake Hr').drop('% of planned output')


# calculate correlation coefficient between all cols
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        col1 = cols[i]
        col2 = cols[j]
        # try:
        # corr, p = pearsonr(df[col1].dropna(), df[col2].dropna())
        # except:
        corr = df[col1].corr(df[col2])
        # p=None
        highest[col1 + '-' + col2] = f'{corr}'#, {p}'

highest = sorted(highest.items(), key=lambda x: x[1], reverse=True)
for k, v in highest:
    print(k, v, sep=':\t')


overall-happiness:	0.8951773030444721
happiness-excitement:	0.7222670670520442
overall-health:	0.7190140429131078
health-positivity:	0.7182412683488743
overall-excitement:	0.6952100812655261
health-happiness:	0.6733042284840807
overall-positivity:	0.6664853902201708
happiness-positivity:	0.6597862601333967
overall-sociability:	0.6315261881311705
integrity-purpose:	0.6215190484408324
health-moment:	0.6009568200242212
positivity-moment:	0.599453450050934
sociability-happiness:	0.5988704776824503
health-energy:	0.5967354745951695
sociability-moment:	0.5885954042114805
overall-moment:	0.5744434608688106
uniqueness-excitement:	0.5659181810381541
sociability-positivity:	0.5633588115041812
energy-excitement:	0.5551438239214108
positivity-integrity:	0.5446788849253412
happiness-moment:	0.544201654650291
moment-integrity:	0.5364882172442437
overall-energy:	0.5261207810259757
happiness-energy:	0.512283733077104
energy-positivity:	0.49115055863837925
positivity-excitement:	0.48662164352428544
soc